In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count,sum,avg,min,mean,max

from pyspark.sql.functions import col,lit,udf
from pyspark.sql.types import IntegerType, DoubleType

In [0]:
spark = SparkSession.builder.appName('Project').getOrCreate()

In [0]:
df = spark.read.options(inferSchema=True,header=True,delimeter=',').csv('/FileStore/tables/OfficeDataProject.csv')
df.show()

+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

In [0]:
df.count()

Out[4]: 1000

In [0]:
df.select('department').distinct().count()

Out[15]: 6

In [0]:
df.select('department').distinct().show()

+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



In [0]:
df.groupBy('department').count().show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|  169|
|        HR|  171|
|   Finance|  162|
|Purchasing|  166|
| Marketing|  170|
|  Accounts|  162|
+----------+-----+



In [0]:
df.groupBy('state').count().show()

+-----+-----+
|state|count|
+-----+-----+
|   LA|  205|
|   CA|  205|
|   WA|  208|
|   NY|  173|
|   AK|  209|
+-----+-----+



In [0]:
df.groupBy('state','department').count().show()

+-----+----------+-----+
|state|department|count|
+-----+----------+-----+
|   CA|     Sales|   42|
|   CA| Marketing|   33|
|   NY|  Accounts|   34|
|   NY|     Sales|   27|
|   CA|   Finance|   35|
|   CA|  Accounts|   35|
|   CA|Purchasing|   32|
|   WA|        HR|   47|
|   AK|Purchasing|   30|
|   WA|  Accounts|   27|
|   WA|Purchasing|   38|
|   AK|     Sales|   38|
|   AK|  Accounts|   37|
|   WA| Marketing|   39|
|   LA|        HR|   41|
|   LA|     Sales|   35|
|   AK|        HR|   25|
|   LA|   Finance|   29|
|   AK|   Finance|   37|
|   LA|Purchasing|   45|
+-----+----------+-----+
only showing top 20 rows



In [0]:
df.groupBy('department').agg(min('salary').alias('min'),max('salary').alias('max')).orderBy('max').show()

+----------+----+----+
|department| min| max|
+----------+----+----+
|  Accounts|1007|9890|
|   Finance|1006|9899|
| Marketing|1031|9974|
|        HR|1013|9982|
|     Sales|1103|9982|
|Purchasing|1105|9985|
+----------+----+----+



In [0]:
# when we put multiple condition like inside filter we have to put it in different paranthesis
avg_ny = df.filter(df.state=='NY').groupBy('state').agg(avg('bonus').alias('bonus')).select('bonus').collect()[0]['bonus']
df.filter((df.state=='NY') & (df.bonus>=avg_ny)).show()

+-----------+--------------------+----------+-----+------+---+-----+
|employee_id|       employee_name|department|state|salary|age|bonus|
+-----------+--------------------+----------+-----+------+---+-----+
|       1014|       Marvis Cobian|Purchasing|   NY|  5061| 41| 1765|
|       1030|          Trena Benz|  Accounts|   NY|  4376| 49| 1624|
|       1035|       Vivan Sifford|   Finance|   NY|  1129| 35| 1261|
|       1042|        Jaclyn Layla|        HR|   NY|  7541| 25| 1289|
|       1044|         Escoto Kohn|Purchasing|   NY|  2268| 44| 1533|
|       1045|   Kaczorowski Dynes|Purchasing|   NY|  3901| 42| 1802|
|       1054|    Lisabeth Gallman| Marketing|   NY|  2143| 37| 1623|
|       1059|       Suzanne Gilma|Purchasing|   NY|  2092| 37| 1840|
|       1065|          Luisa Jere|     Sales|   NY|  7870| 36| 1866|
|       1073|      Herder Gallman|   Finance|   NY|  1988| 31| 1402|
|       1077|        Vivan Locust|        HR|   NY|  8389| 34| 1313|
|       1082|          Nena Rocha|

+-----------+--------------------+----------+-----+------+---+-----+
|employee_id|       employee_name|department|state|salary|age|bonus|
+-----------+--------------------+----------+-----+------+---+-----+
|       1014|       Marvis Cobian|Purchasing|   NY|  5061| 41| 1765|
|       1030|          Trena Benz|  Accounts|   NY|  4376| 49| 1624|
|       1035|       Vivan Sifford|   Finance|   NY|  1129| 35| 1261|
|       1042|        Jaclyn Layla|        HR|   NY|  7541| 25| 1289|
|       1044|         Escoto Kohn|Purchasing|   NY|  2268| 44| 1533|
|       1045|   Kaczorowski Dynes|Purchasing|   NY|  3901| 42| 1802|
|       1054|    Lisabeth Gallman| Marketing|   NY|  2143| 37| 1623|
|       1059|       Suzanne Gilma|Purchasing|   NY|  2092| 37| 1840|
|       1065|          Luisa Jere|     Sales|   NY|  7870| 36| 1866|
|       1073|      Herder Gallman|   Finance|   NY|  1988| 31| 1402|
|       1077|        Vivan Locust|        HR|   NY|  8389| 34| 1313|
|       1082|          Nena Rocha|

In [0]:
def func(age,currentSalary):
    if age>=45:
        return currentSalary+500
    return currentSalary
incSalUDF = udf(lambda x,y: func(x,y),IntegerType())
df.withColumn('salary',incSalUDF(df.age,df.salary)).show()


+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  6217| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  4204| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

In [0]:
df.filter(df.age>=45).write.csv('/FileStore/tables/mini_project_saved_1.csv')